In [1]:
import pandas as pd

df = pd.read_parquet("df_english_filtered_articles.cache.parquet")

print(df.shape)

df.head(2)

(50000, 21)


,page_id,title,url,word_count,outlink_count,category_count,categories,template_count,external_link_count,last_modified,...,title_match,total_views,aggregated_page_ids,url_pageviews,views_percentile,rn,chunk_size,split_text,n_chunks,n_words_per_chunk
7272,1249296276,2000s,https://en.wikipedia.org/wiki/2000s,33738,3131,5,Category:2000s|Category:20th century|Category:...,571,866,2024-10-04 05:23:25,...,2000s,293131.0,"[36397, null, 19671240, 34579],[null, 607345, ...",https://no.wikipedia.org/w/index.php?curid=124...,0.099228,1,1318,"[File:2000s decade montage3.png|From top left,...",229,"[227, 182, 197, 128, 131, 157, 198, 191, 191, ..."
6937,47999371,Contract,https://en.wikipedia.org/wiki/Contract,27356,801,2,Category:Contract law|Category:Legal documents,232,264,2024-09-30 13:08:35,...,contract,306356.0,"[19280537],[19280537, null],[null, 19280537]",https://no.wikipedia.org/w/index.php?curid=479...,0.094670,1,1240,[A contract is an agreement that specifies cer...,208,"[98, 114, 173, 46, 103, 145, 134, 134, 90, 151..."


In [2]:
processed_text = df.processed_text.iloc[0]

print(processed_text.strip()[:100])

File:2000s decade montage3.png|From top left, clockwise: The World Trade Center on fire and the Stat


In [25]:
# check rows with the same page_id
df[df.page_id.duplicated()].sort_values("page_id").head(3)

,page_id,title,url,word_count,outlink_count,category_count,categories,template_count,external_link_count,last_modified,processed_text,title_match,total_views,aggregated_page_ids,url_pageviews,views_percentile,rn,chunk_size,split_text,n_chunks,n_words_per_chunk,title_normalized
5608,2,Rilmenidine,https://en.wikipedia.org/wiki/Rilmenidine,625,26,4,Category:Alpha-2 adrenergic receptor agonists|...,21,1,2024-09-11 19:54:24,\n\n\nRilmenidine is a prescription medication...,rilmenidine,362989.0,"[null, 7512833],[7512833]",https://no.wikipedia.org/w/index.php?curid=2,0.076687,1,1865,[Rilmenidine is a prescription medication for ...,3,"[216, 217, 223]",rilmenidine
20307,2,Cuttlebone,https://en.wikipedia.org/wiki/Cuttlebone,1063,72,4,Category:Articles containing video clips|Categ...,34,14,2024-06-29 19:06:37,\n\n\n\n\n\nthumb|Fossil cuttlebone of the Pli...,cuttlebone,103626.0,"[5720791, 2333993],[2333993],[null, 2333993]",https://no.wikipedia.org/w/index.php?curid=2,0.275118,1,1776,[thumb|Fossil cuttlebone of the Pliocene speci...,5,"[262, 258, 108, 230, 227]",cuttlebone
28982,76,W76,https://en.wikipedia.org/wiki/W76,715,26,2,Category:Nuclear warheads of the United States...,22,24,2024-06-18 12:12:02,\n\n\n\nThe W76 is an American thermonuclear w...,w76,65986.0,"[2307156, 5400347],[2307156],[2307156, null]",https://no.wikipedia.org/w/index.php?curid=76,0.391443,1,1220,"[The W76 is an American thermonuclear warhead,...",6,"[168, 94, 183, 88, 190, 65]",w76


In [8]:
df["title_normalized"] = df.title.str.strip().str.lower().str.replace(" ", "_")
df["title_normalized"].nunique()

50000

## Pick something like 10 articles, just to test the embedding and develop the workflow

In [10]:
pd.set_option("display.max_columns", 100)

df_subset = df.sort_values("total_views", ascending=False).head(10)

print(len(df_subset))

df_subset


10


,page_id,title,url,word_count,outlink_count,category_count,categories,template_count,external_link_count,last_modified,processed_text,title_match,total_views,aggregated_page_ids,url_pageviews,views_percentile,rn,chunk_size,split_text,n_chunks,n_words_per_chunk,title_normalized
0,19404073,--,https://en.wikipedia.org/wiki/--,100,12,0,,3,0,2024-07-16 16:39:45,"\n--, a string of two hyphen-minus characters,...",,339535996.0,"[1452809, 68713497, 1546216, 907691, 12686647,...",https://no.wikipedia.org/w/index.php?curid=194...,0.000000,1,1931,"[--, a string of two hyphen-minus characters, ...",1,[97],--
1,299135,YouTube,https://en.wikipedia.org/wiki/YouTube,17507,1244,23,Category:YouTube|Category:2005 establishments ...,802,1029,2024-10-04 20:25:03,\n\n\n\n\n\n\n\nYouTube is an American online...,youtube,61184976.0,"[3524766, 18021924],[3524766, 23602489],[35247...",https://no.wikipedia.org/w/index.php?curid=299135,0.000013,1,1644,[YouTube is an American online video sharing ...,91,"[201, 206, 231, 149, 217, 243, 239, 172, 101, ...",youtube
2,48474643,Cleopatra,https://en.wikipedia.org/wiki/Cleopatra,17807,1109,22,Category:Cleopatra|Category:69 BC births|Categ...,1398,140,2024-10-04 04:20:49,\n\n\n\n\n\n\n\n\n\n\nCleopatra VII Thea Philo...,cleopatra,55414365.0,"[60827],[60827, null, 1359777],[null, 57659752...",https://no.wikipedia.org/w/index.php?curid=484...,0.000026,1,1406,[Cleopatra VII Thea Philopator ( ; The name Cl...,117,"[232, 235, 78, 230, 79, 202, 213, 213, 176, 16...",cleopatra
3,40943757,XXXTentacion,https://en.wikipedia.org/wiki/XXXTentacion,12332,617,38,Category:XXXTentacion|Category:1998 births|Cat...,525,636,2024-09-30 13:11:39,\n\n\n\n\n\nJahseh Dwayne Ricardo Onfroy (Janu...,xxxtentacion,42377950.0,"[53039739],[null, 53039739],[53702987, 5303973...",https://no.wikipedia.org/w/index.php?curid=409...,0.000040,1,1773,"[Jahseh Dwayne Ricardo Onfroy (January 23, 199...",55,"[110, 266, 198, 198, 259, 208, 177, 216, 261, ...",xxxtentacion
4,39604324,Pornhub,https://en.wikipedia.org/wiki/Pornhub,3624,231,10,Category:2007 establishments in Quebec|Categor...,182,277,2024-09-29 18:35:56,\n\n\n\n\nPornhub is a Canadian-owned internet...,pornhub,40291676.0,"[45492650, null, 29555076],[45492650, 32036801...",https://no.wikipedia.org/w/index.php?curid=396...,0.000053,1,1725,[Pornhub is a Canadian-owned internet pornogra...,18,"[205, 239, 228, 198, 189, 197, 244, 241, 205, ...",pornhub
5,46802249,Facebook,https://en.wikipedia.org/wiki/Facebook,17143,1144,26,Category:Facebook|Category:2004 establishments...,917,1027,2024-09-29 08:53:25,\n\n\n\n\n\n\n\n\n\n\n\nFacebook is a social m...,facebook,24563068.0,"[7529378, 5949664],[36114326, 62420226],[75293...",https://no.wikipedia.org/w/index.php?curid=468...,0.000066,1,1126,[Facebook is a social media and social network...,132,"[135, 82, 102, 144, 151, 150, 135, 174, 157, 1...",facebook
6,38001712,ChatGPT,https://en.wikipedia.org/wiki/ChatGPT,7895,451,7,Category:ChatGPT|Category:2022 software|Catego...,334,515,2024-10-05 13:15:20,\n\n\n\n\n\n\nChatGPT is a generative artifici...,chatgpt,21330575.0,"[72417803, 73128493],[72822812, 72417803],[nul...",https://no.wikipedia.org/w/index.php?curid=380...,0.000079,1,1938,[ChatGPT is a generative artificial intelligen...,33,"[202, 277, 213, 260, 297, 269, 261, 270, 256, ...",chatgpt
7,163732,.xxx,https://en.wikipedia.org/wiki/.xxx,1195,44,4,Category:Internet properties established in 20...,40,67,2024-08-30 02:40:52,"\n\n\n\n\n\n.xxx (pronounced ""dot triple-ecks""...",.xxx,17627534.0,"[null, 574533],[574533],[1984564, 574533]",https://no.wikipedia.org/w/index.php?curid=163732,0.000092,1,1597,"[.xxx (pronounced ""dot triple-ecks"" or ""dot ec...",6,"[225, 214, 178, 227, 214, 210]",.xxx
8,45583799,Sex,https://en.wikipedia.org/wiki/Sex,3431,298,2,Category:Sex|Category:Biological processes,164,56,2024-08-30 11:51:23,\n\n\n\n\n\n\n\n\nSex is the biological trait ...,sex,17502835.0,"[1093432, 957715],[1093432, 21260215],[131886]...",https://no.wikip

In [17]:
print(df_subset.columns)

# df_subset[:2].to_dict(orient="records")

Index(['page_id', 'title', 'url', 'word_count', 'outlink_count',
       'category_count', 'categories', 'template_count', 'external_link_count',
       'last_modified', 'processed_text', 'title_match', 'total_views',
       'aggregated_page_ids', 'url_pageviews', 'views_percentile', 'rn',
       'split_text', 'n_chunks', 'n_words_per_chunk', 'title_normalized'],
      dtype='object')


In [18]:
# df_subset.iloc[2].to_json()
#

In [11]:
df_subset.n_chunks.sum()

580

In [13]:
df_subset.iloc[1:2].T


,1
page_id,299135
title,YouTube
url,https://en.wikipedia.org/wiki/YouTube
word_count,17507
outlink_count,1244
category_count,23
categories,Category:YouTube|Category:2005 establishments ...
template_count,802
external_link_count,1029
last_modified,2024-10-04 20:25:03


In [14]:
"""
Here we pick out both the data we need to embed, 
And the metadata we want to store in pinecone.

In the end we return a dataframe, which can be used to insert data 
into an SQL database to match what is in the vector db.
"""

prefix_template = "This is text from the wikipedia article with title: {title}"

data = []

for idx, row in df_subset.iterrows():
    # Add each chunk of text with its metadata
    # for i, chunk in enumerate(row['split_text']):
    for i in range(row["n_chunks"]):
        # prefix each chunk with the title of the article
        chunk_text = row["split_text"][i]
        prefix = prefix_template.format(title=row["title"].strip())
        chunk_text = f"{prefix}\n{chunk_text}"

        data.append(
            {
                "chunk_idx": i,
                "chunk_text": chunk_text,
                "page_id": str(row["page_id"]),
                "title": row["title"].strip(),
                "url": row["url"],
                "title_match": row["title_match"],
                "title_normalized": row["title_normalized"],
                "word_count_total": row["word_count"],
                "word_count_chunk": row["n_words_per_chunk"][i],
                "chunk_size": row["chunk_size"],
            }
        )


df_to_embed = pd.DataFrame(data)

print(len(df_to_embed))
df_to_embed.head(2)


580


,chunk_idx,chunk_text,page_id,title,url,title_match,title_normalized,word_count_total,word_count_chunk,chunk_size
0,0,This is text from the wikipedia article with t...,19404073,--,https://en.wikipedia.org/wiki/--,,--,100,97,1931
1,0,This is text from the wikipedia article with t...,299135,YouTube,https://en.wikipedia.org/wiki/YouTube,youtube,youtube,17507,201,1644


In [16]:
import pandas as pd
import os
from pinecone import Pinecone, ServerlessSpec

INDEX_NAME = "english-wiki"
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")

pc = Pinecone(api_key=PINECONE_API_KEY)


# Define batch size
BATCH_SIZE = 50

# Initialize list to store all embeddings
all_embeddings = []

# Iterate over the DataFrame in batches
for start_idx in range(0, len(df_to_embed), BATCH_SIZE):
    end_idx = start_idx + BATCH_SIZE
    batch_texts = df_to_embed["chunk_text"].iloc[start_idx:end_idx].tolist()

    # Generate embeddings for the current batch
    batch_embeddings = pc.inference.embed(
        model="multilingual-e5-large",
        inputs=batch_texts,
        parameters={"input_type": "passage", "truncate": "END"},
    )

    # Append the batch embeddings to the main list
    all_embeddings.extend(batch_embeddings)

    print(
        f"Processed batch {start_idx // BATCH_SIZE + 1}: Embeddings {start_idx} to {end_idx - 1}"
    )

# Verify the number of embeddings matches the DataFrame
assert len(all_embeddings) == len(
    df_to_embed
), "Mismatch between embeddings and DataFrame rows"


Processed batch 1: Embeddings 0 to 49
Processed batch 2: Embeddings 50 to 99
Processed batch 3: Embeddings 100 to 149
Processed batch 4: Embeddings 150 to 199
Processed batch 5: Embeddings 200 to 249
Processed batch 6: Embeddings 250 to 299
Processed batch 7: Embeddings 300 to 349
Processed batch 8: Embeddings 350 to 399
Processed batch 9: Embeddings 400 to 449
Processed batch 10: Embeddings 450 to 499
Processed batch 11: Embeddings 500 to 549
Processed batch 12: Embeddings 550 to 599


In [17]:
# Add the embeddings to the DataFrame
embedding_values = [e["values"] for e in all_embeddings]
df_to_embed["embedding_vector"] = embedding_values

print(f"Total embeddings added: {len(df_to_embed)}")
df_to_embed.head(2)

Total embeddings added: 580


,chunk_idx,chunk_text,page_id,title,url,title_match,title_normalized,word_count_total,word_count_chunk,chunk_size,embedding_vector
0,0,This is text from the wikipedia article with t...,19404073,--,https://en.wikipedia.org/wiki/--,,--,100,97,1931,"[0.03192138671875, -0.011444091796875, -0.0243..."
1,0,This is text from the wikipedia article with t...,299135,YouTube,https://en.wikipedia.org/wiki/YouTube,youtube,youtube,17507,201,1644,"[0.014129638671875, 0.0012607574462890625, -0...."


In [19]:
"""
Prepare data to be stored in the vector db (pinecone).

Note that "id" and "values" are the required fields for pinecone. 
Whatever we want to store in addition to the required fields  
needs to be stored in the "metadata" field.
"""

df_to_embed["vector_id"] = (
    df_to_embed["title_normalized"] + "_" + df_to_embed["chunk_idx"].astype(str)
)

vector_data = []

for idx, row in df_to_embed.iterrows():
    vector_data.append(
        {
            "id": row["vector_id"],
            "values": row["embedding_vector"],
            "metadata": {
                "title": row["title"],
                "url": row["url"],
                "chunk_idx": row["chunk_idx"],
                "chunk_text": row["chunk_text"],
            },
        }
    )

pd.DataFrame(vector_data[:2])


,id,values,metadata
0,--_0,"[0.03192138671875, -0.011444091796875, -0.0243...","{'title': '--', 'url': 'https://en.wikipedia.o..."
1,youtube_0,"[0.014129638671875, 0.0012607574462890625, -0....","{'title': 'YouTube', 'url': 'https://en.wikipe..."


In [20]:
index = pc.Index(INDEX_NAME)

# index.delete(delete_all=True)
# print(f"All vectors have been successfully deleted from the index '{INDEX_NAME}'.")

# # Upsert in batches of 100
batch_size = 100
for i in range(0, len(vector_data), batch_size):
    batch = vector_data[i : i + batch_size]
    index.upsert(vectors=batch)

print(f"Upserted {len(vector_data)} vectors to the index '{INDEX_NAME}'.")


All vectors have been successfully deleted from the index 'english-wiki'.
Upserted 580 vectors to the index 'english-wiki'.


In [22]:
# df_to_embed = df_to_embed.drop(columns=["embedding"])

In [23]:
df_to_embed.to_parquet("df_to_embed_english_subset_10_articles_finished.parquet")

In [24]:
df_to_embed.title.unique()

array(['--', 'YouTube', 'Cleopatra', 'XXXTentacion', 'Pornhub',
       'Facebook', 'ChatGPT', '.xxx', 'Sex', 'Instagram'], dtype=object)

## Test the WikiEmbedder 
**(It should have the functionality of this notebook, witch checkpointing)**



In [28]:
index.delete(delete_all=True)
print(f"All vectors have been successfully deleted from the index '{INDEX_NAME}'.")


All vectors have been successfully deleted from the index 'english-wiki'.


In [29]:
df.head(10).to_parquet("df_test_wiki_embedder.parquet")


In [37]:
import sys
from importlib import reload

sys.path.append("../wikipedia_embeddings_clean/")

import wikipedia_embedder
reload(wikipedia_embedder)

from wikipedia_embedder import WikiEmbedder



input_parquet_path = "df_test_wiki_embedder.parquet"
checkpoint_path = "checkpoint_test_wiki_embedder.parquet"
pinecone_index_name = "english-wiki"
# batch size controls how many we upsert to pinecone at a time
# and also how many are embedded at a time.
# something like 50 is a good number.
batch_size = 50
embedding_model = "multilingual-e5-large"

wiki_embedder = WikiEmbedder(
    input_parquet_path,
    checkpoint_path,
    pinecone_index_name,
    batch_size=batch_size,
    embedding_model=embedding_model,
)

# controls how many articles are processed at a time
articles_per_batch = 3

wiki_embedder.process_articles()

2024-12-07 14:46:59,779 - INFO - Discovering subpackages in _NamespacePath(['/Users/einar/git/hafsteinn/together_rag/venv/lib/python3.10/site-packages/pinecone_plugins'])
2024-12-07 14:46:59,781 - INFO - Looking for plugins in pinecone_plugins.inference
2024-12-07 14:46:59,781 - INFO - Installing plugin inference into Pinecone
2024-12-07 14:46:59,792 - INFO - Processing articles 0 to 9
2024-12-07 14:47:00,123 - INFO - No new chunks to process in this batch


In [33]:
df_input = pd.read_parquet(input_parquet_path)
print(df_input.shape)
df_input.head(2)


(10, 22)


,page_id,title,url,word_count,outlink_count,category_count,categories,template_count,external_link_count,last_modified,processed_text,title_match,total_views,aggregated_page_ids,url_pageviews,views_percentile,rn,chunk_size,split_text,n_chunks,n_words_per_chunk,title_normalized
7272,1249296276,2000s,https://en.wikipedia.org/wiki/2000s,33738,3131,5,Category:2000s|Category:20th century|Category:...,571,866,2024-10-04 05:23:25,\n\n\n\nFile:2000s decade montage3.png|From to...,2000s,293131.0,"[36397, null, 19671240, 34579],[null, 607345, ...",https://no.wikipedia.org/w/index.php?curid=124...,0.099228,1,1318,"[File:2000s decade montage3.png|From top left,...",229,"[227, 182, 197, 128, 131, 157, 198, 191, 191, ...",2000s
6937,47999371,Contract,https://en.wikipedia.org/wiki/Contract,27356,801,2,Category:Contract law|Category:Legal documents,232,264,2024-09-30 13:08:35,\n\n\n\n\nA contract is an agreement that spec...,contract,306356.0,"[19280537],[19280537, null],[null, 19280537]",https://no.wikipedia.org/w/index.php?curid=479...,0.094670,1,1240,[A contract is an agreement that specifies cer...,208,"[98, 114, 173, 46, 103, 145, 134, 134, 90, 151...",contract


In [34]:
df_input.n_chunks.sum()

1865

In [36]:
df_checkpoint = pd.read_parquet(checkpoint_path)

print(df_checkpoint.shape)
df_checkpoint.head(2)

(1865, 2)


,title_match,chunk_idx
0,2000s,0
1,2000s,1
